In [3]:
# Transformers installation
# ! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

```bash
pip install transformers datasets evaluate -q
pip install jiwer -q
```

# 1. Imports 

In [4]:
import PIL.Image
import torchvision
import io
import requests
import datasets
import transformers
from transformers import TrainingArguments, Trainer
import torch
from transformers import AutoProcessor

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Datasets

In [5]:
# Conceptual Captions 데이터셋 불러오기
ds = datasets.load_dataset("conceptual_captions", split="train")
ds = ds.train_test_split(test_size=0.3)
train_ds = ds["train"]
test_ds = ds["test"]

In [7]:
train_ds[0]

{'image_url': 'https://ak3.picdn.net/shutterstock/videos/6783403/thumb/1.jpg',
 'caption': 'skier loses his balance in the air after pulling a flip , lands heavily in to deep snow'}

In [8]:
examples_dct = train_ds[:5]

In [9]:
examples_dct['image_url']

['https://ak3.picdn.net/shutterstock/videos/6783403/thumb/1.jpg',
 'https://ak7.picdn.net/shutterstock/videos/7829737/thumb/1.jpg?i10c=img.resize(height:160)',
 'https://ak3.picdn.net/shutterstock/videos/17882833/thumb/1.jpg',
 'https://media.istockphoto.com/photos/smartphone-with-isolated-white-screen-with-empty-space-for-text-in-a-picture-id690390258?k=6&m=690390258&s=612x612&w=0&h=za0OqsRcOAFFqhpHbVALLnZbrpa9m40MlGnvwjaOC2k=',
 'http://i.dailymail.co.uk/i/pix/2011/03/07/article-1363727-0D80B5F0000005DC-693_634x747.jpg']

In [10]:
def URL을PIL이미지로변환하기(url):
    img = PIL.Image.open(io.BytesIO(requests.get(url).content))
    return img

In [17]:
URL을PIL이미지로변환하기(train_ds[5]['image_url'])

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x71e5abcc9da0>

In [23]:
train_ds[:6]['image_url']

['https://ak3.picdn.net/shutterstock/videos/6783403/thumb/1.jpg',
 'https://ak7.picdn.net/shutterstock/videos/7829737/thumb/1.jpg?i10c=img.resize(height:160)',
 'https://ak3.picdn.net/shutterstock/videos/17882833/thumb/1.jpg',
 'https://media.istockphoto.com/photos/smartphone-with-isolated-white-screen-with-empty-space-for-text-in-a-picture-id690390258?k=6&m=690390258&s=612x612&w=0&h=za0OqsRcOAFFqhpHbVALLnZbrpa9m40MlGnvwjaOC2k=',
 'http://i.dailymail.co.uk/i/pix/2011/03/07/article-1363727-0D80B5F0000005DC-693_634x747.jpg',
 'https://media.gettyimages.com/photos/martin-gore-of-depeche-mode-performs-live-on-stage-during-the-delta-picture-id450620313']

In [24]:
URL을PIL이미지로변환하기('https://media.gettyimages.com/photos/martin-gore-of-depeche-mode-performs-live-on-stage-during-the-delta-picture-id450620313')

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x71e5ad537290>

## Preprocess the dataset

Since the dataset has two modalities (image and text), the pre-processing pipeline will preprocess images and the captions.

To do so, load the processor class associated with the model you are about to fine-tune.

In [235]:
모델이름 = "microsoft/git-base"
전처리하기1 = AutoProcessor.from_pretrained(모델이름)

`-` `전처리하기1` 사용방법

In [236]:
#전처리하기1(images=[img,img])

`-` 전처리하기2

In [1]:
def URL을PIL이미지로변환하기(url):
    img = PIL.Image.open(io.BytesIO(requests.get(url).content))
    return img
def 전처리하기2(examples_dct):
    PIL이미지리스트 = [URL을PIL이미지로변환하기(url) for url in examples_dct['image_url']]
    캡션리스트 = [x for x in examples_dct["caption"]]
    전처리된자료 = 전처리하기1(images=PIL이미지리스트, text=캡션리스트, padding="max_length")
    return 전처리된자료
train전처리된자료 = train_ds.with_transform(전처리하기2)
test전처리된자료 = test_ds.with_transform(전처리하기2)

NameError: name 'train_ds' is not defined

In [25]:
def filter_valid_images(example):
    url = example["image_url"]
    try:
        URL을PIL이미지로변환하기(url)
        # URL 응답 확인
        return True
    except:
        return False 

In [1]:
train_ds.select(range(100)).filter(filter_valid_images)

NameError: name 'train_ds' is not defined

In [252]:
train_ds.filter(filter_valid_images)

Filter:   0%|          | 0/2986499 [00:00<?, ? examples/s]

In [196]:
_데이터콜렉터입력샘플[:1


SyntaxError: invalid syntax. Perhaps you forgot a comma? (97606739.py, line 3)

In [ ]:
model_input_sample = dict(
    input_ids = torch.tensor(train_ds_transformed[:2]['input_ids']),
    attention_mask = torch.tensor(train_ds_transformed[:2]['attention_mask']),
    pixel_values = torch.tensor(train_ds_transformed[:2]['pixel_values'])
)

In [141]:
trainer = Trainer(
    model=model,
    #args=training_args,
    train_dataset=train_ds,
    #eval_dataset=test_ds,
    #compute_metrics=compute_metrics,
)

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /opt/conda/conda-bld/pytorch_1720538439675/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [161]:
train_ds_transformed[:2].keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])

In [171]:
model_input_sample = dict(
    input_ids = torch.tensor(train_ds_transformed[:2]['input_ids']),
    attention_mask = torch.tensor(train_ds_transformed[:2]['attention_mask']),
    pixel_values = torch.tensor(train_ds_transformed[:2]['pixel_values'])
)

In [172]:
model(**model_input_sample)

CausalLMOutputWithPast(loss=None, logits=tensor([[[-0.8686, -0.8685, -0.8684,  ..., -0.8684, -0.8684, -0.8684],
         [-0.5816, -0.5817, -0.5818,  ..., -0.5815, -0.5818, -0.5812],
         [-0.7298, -0.7297, -0.7296,  ..., -0.7297, -0.7296, -0.7297],
         ...,
         [-2.0791, -2.0784, -2.0771,  ..., -2.0789, -2.0774, -2.0801],
         [-1.7354, -1.7346, -1.7330,  ..., -1.7352, -1.7332, -1.7364],
         [-2.1231, -2.1235, -2.1239,  ..., -2.1229, -2.1234, -2.1227]],

        [[-0.5791, -0.5787, -0.5777,  ..., -0.5791, -0.5781, -0.5796],
         [ 0.1308,  0.1317,  0.1333,  ...,  0.1309,  0.1326,  0.1299],
         [-1.4053, -1.4058, -1.4065,  ..., -1.4052, -1.4061, -1.4047],
         ...,
         [-1.6593, -1.6588, -1.6575,  ..., -1.6593, -1.6578, -1.6602],
         [-1.7329, -1.7321, -1.7304,  ..., -1.7328, -1.7308, -1.7340],
         [-1.8127, -1.8122, -1.8111,  ..., -1.8125, -1.8112, -1.8135]]],
       grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[ 1.9375e-01, 

In [121]:
[len(l) for l in _input_ids]

[512, 512]

In [75]:
[url2tensor(url) for url in examples_dct['image_url']]

['https://static1.squarespace.com/static/5864a522d1758e09281c7249/t/588567d3ebbd1a30e993ac41/1485138873686/?format=750w',
 'https://ak1.picdn.net/shutterstock/videos/24875111/thumb/1.jpg',
 'https://media.gettyimages.com/photos/actor-lee-ming-shun-and-actress-bianca-bai-attend-a-commercial-of-picture-id595911976?s=612x612',
 'https://us.123rf.com/450wm/jpgon/jpgon1704/jpgon170401613/76795298-illustration-of-an-isolated-female-head-with-a-lotus-flower.jpg?ver=6',
 'https://media.gettyimages.com/photos/clyde-during-a-military-exercise-to-show-the-governor-its-boats-picture-id138569967']

In [78]:
url2tensor??

Signature: url2tensor(url)
Docstring: <no docstring>
Source:   
def url2tensor(url):
    url = train_ds[0]['image_url']
    img = PIL.Image.open(io.BytesIO(requests.get(url).content))
    return 이미지전처리하기(img)
File:      /tmp/ipykernel_1414278/2182745578.py
Type:      function

In [ ]:
examples_dct['image_url']

In [71]:
test_ds_transformed[0]

RecursionError: maximum recursion depth exceeded

With the dataset ready, you can now set up the model for fine-tuning.

## Load a base model

Load the ["microsoft/git-base"](https://huggingface.co/microsoft/git-base) into a [`AutoModelForCausalLM`](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForCausalLM) object.

In [45]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [54]:
model??

Signature:      model(*args, **kwargs)
Type:           GitForCausalLM
String form:   
GitForCausalLM(
  (git): GitModel(
    (embeddings): GitEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (image_encoder): GitVisionModel(
      (vision_model): GitVisionTransformer(
        (embeddings): GitVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
          (position_embedding): Embedding(197, 768)
        )
        (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (encoder): GitVisionEncoder(
          (layers): ModuleList(
            (0-11): 12 x GitVisionEncoderLayer(
              (self_attn): GitVisionAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
      

## Evaluate

Image captioning models are typically evaluated with the [Rouge Score](https://huggingface.co/spaces/evaluate-metric/rouge) or [Word Error Rate](https://huggingface.co/spaces/evaluate-metric/wer). For this guide, you will use the Word Error Rate (WER).

We use the 🤗 Evaluate library to do so. For potential limitations and other gotchas of the WER, refer to [this guide](https://huggingface.co/spaces/evaluate-metric/wer).

In [ ]:
from evaluate import load
import torch

wer = load("wer")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predicted = logits.argmax(-1)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)
    decoded_predictions = processor.batch_decode(predicted, skip_special_tokens=True)
    wer_score = wer.compute(predictions=decoded_predictions, references=decoded_labels)
    return {"wer_score": wer_score}

## Train!

Now, you are ready to start fine-tuning the model. You will use the 🤗 [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) for this.

First, define the training arguments using [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments).

In [ ]:
from transformers import TrainingArguments, Trainer

model_name = checkpoint.split("/")[1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-pokemon",
    learning_rate=5e-5,
    num_train_epochs=50,
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    save_total_limit=3,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    logging_steps=50,
    remove_unused_columns=False,
    push_to_hub=True,
    label_names=["labels"],
    load_best_model_at_end=True,
)

Then pass them along with the datasets and the model to 🤗 Trainer.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

To start training, simply call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) on the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) object.

In [ ]:
trainer.train()

You should see the training loss drop smoothly as training progresses.

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [ ]:
trainer.push_to_hub()

## Inference

Take a sample image from `test_ds` to test the model.

In [ ]:
from PIL import Image
import requests

url = "https://huggingface.co/datasets/sayakpaul/sample-datasets/resolve/main/pokemon.png"
image = Image.open(requests.get(url, stream=True).raw)
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/test_image_image_cap.png" alt="Test image"/>
</div>
    
Prepare image for the model.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

inputs = processor(images=image, return_tensors="pt").to(device)
pixel_values = inputs.pixel_values

Call `generate` and decode the predictions.

In [ ]:
generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)

```bash
a drawing of a pink and blue pokemon
```

Looks like the fine-tuned model generated a pretty good caption!